## CHATBOT

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth',-1) # ini untuk melihat full column
client = MongoClient('localhost', 27017) #ihi host
db=client.get_database('fujiatma') # ini nama database sebelumnya create dlu di mongodb atau di robomongo
collection=db.get_collection('data')# ini nama collectionnya (misal data)
#collection.find_one({ "question" : "Does it fit Nook GlowLight?"})# ini contoh query saja


#cara dump ke mongo
#mongoimport --db fujiatma --collection data --file intents.json

In [2]:
cursor=collection.find() # query untuk melihat isi collection
data = pd.DataFrame(list(collection.find())) # merubah ke data frame
#data=data[data['question']!=np.nan]
#data=data.dropna(axis=0, how='all')
data.head()

,_id,answer,answerTime,answerType,asin,intents,question,questionType,unixTime
0,5a6b45346337ffb0d53c8710,Yes this fits both the nook color and the same-shaped nook tablet,"Dec 27, 2013",Y,0594033926,NaN,Is this cover the one that fits the old nook color? Which I believe is 8x5.,yes/no,1.388131e+09
1,5a6b45346337ffb0d53c8711,No. The nook color or color tablet,"Jan 5, 2015",N,0594033926,NaN,Does it fit Nook GlowLight?,yes/no,1.420445e+09
2,5a6b45346337ffb0d53c8712,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close.",2 days ago,NaN,0594033926,NaN,Would it fit Nook 1st Edition? 4.9in x 7.7in ?,open-ended,NaN
3,5a6b45346337ffb0d53c8713,yes,17 days ago,Y,0594033926,NaN,Will this fit a Nook Color that's 5 x 8?,yes/no,NaN
4,5a6b45346337ffb0d53c8714,"No, the tab is smaller than the 'color'","Feb 10, 2015",N,0594033926,NaN,will this fit the Samsung Galaxy Tab 4 Nook 10.1,yes/no,1.423555e+09


In [3]:
data_ques_answer=data[['question','answer']][:27000]
print "banyak data:", len(data_ques_answer)
data_ques_answer.head()

banyak data: 27000


,question,answer
0,Is this cover the one that fits the old nook color? Which I believe is 8x5.,Yes this fits both the nook color and the same-shaped nook tablet
1,Does it fit Nook GlowLight?,No. The nook color or color tablet
2,Would it fit Nook 1st Edition? 4.9in x 7.7in ?,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close."
3,Will this fit a Nook Color that's 5 x 8?,yes
4,will this fit the Samsung Galaxy Tab 4 Nook 10.1,"No, the tab is smaller than the 'color'"


In [4]:
# Preproses: proses jika di jadikan satu
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
porter_stemmer = PorterStemmer()

def preprocess(corpus):
    #melakukan proses lowercase, stopword removal, dan punctuation removal
    print "proses lowercase"
    m=[]

    for item in range(len(corpus)):
        text=string.lower(corpus['question'].iloc[item])
        #jawab=string.lower(corpus['answer'].iloc[item])
        corpus['question'].iloc[item]=text  
        #corpus['answer'].iloc[item]=jawab

    print "proses stopwords removal"
    cachedStopWords = set(stopwords.words("english"))
    for i in range(len(corpus)):
        sent=corpus['question'].iloc[i]
        kt=" ".join([word for word in sent.split() if word not in cachedStopWords])
        corpus['question'].iloc[i]=kt
        
    print "proses punctuation"
    remove=string.punctuation

    for i in range(len(corpus)):
        sent=data_ques_answer['question'].iloc[i]
        kd=' '.join(word.strip(remove) for word in sent.split())
        corpus['question'].iloc[i]=kd
    return corpus

In [5]:
data_normal=preprocess(data_ques_answer)
print 'jumlah data:',len(data_normal)
data_normal.head()

proses lowercase
proses stopwords removal
proses punctuation
jumlah data: 27000


,question,answer
0,cover one fits old nook color believe 8x5,Yes this fits both the nook color and the same-shaped nook tablet
1,fit nook glowlight,No. The nook color or color tablet
2,would fit nook 1st edition 4.9in x 7.7in,"I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close."
3,fit nook color that's 5 x 8,yes
4,fit samsung galaxy tab 4 nook 10.1,"No, the tab is smaller than the 'color'"


## PENDEKATAN NAIVE BAYES, SVM, MLP [n_data 100]

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [8]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(data_normal.question[:100])
X_train_counts.shape

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape



tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape
data_normal.question[:5]

0    cover one fits old nook color believe 8x5
1    fit nook glowlight                       
2    would fit nook 1st edition 4.9in x 7.7in 
3    fit nook color that's 5 x 8              
4    fit samsung galaxy tab 4 nook 10.1       
Name: question, dtype: object

In [11]:
clf_nb = MultinomialNB().fit(X_train_tfidf, data_normal.answer[:100])
clf_svm = LinearSVC().fit(X_train_tfidf, data_normal.answer[:100])
clf_mlp = MLPClassifier().fit(X_train_tfidf, data_normal.answer[:100])
data_normal.answer[:5]

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


0    Yes this fits both the nook color and the same-shaped nook tablet                                               
1    No. The nook color or color tablet                                                                              
2    I don't think so. The nook color is 5 x 8 so not sure anything smaller would stay locked in, but would be close.
3    yes                                                                                                             
4    No, the tab is smaller than the 'color'                                                                         
Name: answer, dtype: object

In [13]:
print clf_nb.score(X_train_tfidf, data_normal.answer[:100])
print clf_svm.score(X_train_tfidf, data_normal.answer[:100])
print clf_mlp.score(X_train_tfidf, data_normal.answer[:100])

0.87
0.99
0.99


In [22]:
docs_new = 'good morning'
print 'pertanyaan:\n',docs_new
docs_new=string.lower(docs_new)
docs_new1=[]
docs_new1.append(docs_new)
docs_new1

X_new_counts = count_vect.transform(docs_new1)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf_mlp.predict(X_new_tfidf)
print "jawab:\n", predicted

pertanyaan:
good morning
jawab:
[u'Yes, you can also use the Amazon Kindle app.']


In [43]:
X_new_counts = count_vect.transform(docs_new1)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
predicted

array([u'fit samsung galaxy tab 4 nook 10.1'], 
      dtype='<U114')

## PENDEKATAN SEQ2SEQ- Code In python 3

In [25]:
'''Sequence to sequence example in Keras (character-level).

This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

# Summary of the algorithm

- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

# Data download

English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip

Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/

# References

- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

print 'ok'
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


SyntaxError: invalid syntax (<ipython-input-25-71d1234573f3>, line 125)